In [1]:
# if you want to have a pop up sign in method instead please uncomment  on ee.Authenticate and ee.Initialize, but comment on below

import ee

# Trigger the authentication flow. if you want to user json, please comment this
ee.Authenticate()
# Initialize the library.
ee.Initialize()

# here we try to use json api information instead (service account) - comment below if you use above methode
#import ee
import math
from datetime import datetime

# uncomment this if you want to use json instead
#service_account = 'iqbalpythonapi@bukit30project.iam.gserviceaccount.com'
#credentials = ee.ServiceAccountCredentials(service_account, './bukit30project-4d92e5b46ea7.json')
#ee.Initialize(credentials)

Enter verification code: 4/1AfJohXlbyewkFyIyLKkCrFSo4FM7D_TwvwC026Nx_BdQ2HKFF75XAcvOq94

Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_doiqkQG3NJ1t8IS?source=API


In [2]:
import geemap

# AOI File (SHP) - added shp_local: please make a folder in the main directory: shp_local and put here as an input (example) - shp local not publish to github
# AOIt_shp = './shp_local/AOI_B30.shp'
AOIt_shp = './shp_local/aoi_b30_smaller.shp'
AOIt_shp_plot = geemap.shp_to_ee(AOIt_shp)
crs_input = 'EPSG:4326'

AOI = AOIt_shp_plot

In [3]:
from backend.sv_carbon_removal.gee_lib.osi.image_collection.image_collection_composite import ImageCollectionComposite

# initiate instance class for the image collection and later mosaicking
classInputCollection = ImageCollectionComposite(AOI=AOI, date_start_end=['2022-1-1',"2022-12-31"], cloud_cover_threshold = 40)


In [4]:
image_landsat_collection_mask = classInputCollection.merging_collection_landsat()

selecting LANDSAT images


In [5]:
image_landsat_cloudless_composite = image_landsat_collection_mask.median().clip(AOI)

In [6]:
Map = geemap.Map(center=(-3, 115), zoom=4)
Map.centerObject(AOI, 10)

In [7]:
# Map.addLayer(image_landsat_cloudless_composite,{'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 0.6, 'gamma': 1.5 }, 'landsat_for_FCD_2023_uncorrected')

In [8]:
image_planet_clip = classInputCollection.merging_collection_planet('asia').median().clip(AOI)

In [9]:
# ADDING VISUAL FOR CHECKING AND VERIFY THE RESULT
visPlanet_veg = {"bands":["red","nir","blue"],"min":0,"max":0.6,"gamma":1.5}
truePlanet_veg = {"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}

# Map.addLayer(image_planet_clip, visPlanet_veg , 'NICFI Cloudless Image - RNB', False)
# Map.addLayer(image_planet_clip, truePlanet_veg , 'NICFI Cloudless Image - RGB')

In [10]:
from osi.image_collection.topographic_correction import doPhysicalCorrection

image_corrected_composite_landsat = image_landsat_collection_mask.map(lambda image: doPhysicalCorrection(image, 
                                                                                                         type_DEM='srtm', 
                                                                                                         I_satellite='LANDSAT')) \
                                                                                                         .median().clip(AOI)

In [11]:
Map.addLayer(image_corrected_composite_landsat,{'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 0.6, 'gamma': 1.5 }, f'landsat_for_FCD_2023_corrected')
    

In [12]:
from backend.sv_carbon_removal.gee_lib.osi.spectral_indices.spectral_analysis import SpectralAnalysis

classImageSpectral = SpectralAnalysis(image_corrected_composite_landsat, AOI = AOI, I_satellite= 'LANDSAT', pca_scaling = 1, tileScale = 1)


In [13]:
avi_image = classImageSpectral.AVI_func()

In [14]:
Map.addLayer(avi_image,{},'avi')

In [15]:
Map.addLayerControl()

In [16]:
seeds = ee.Algorithms.Image.Segmentation.seedGrid(36)

# Run SNIC on the regular square grid. # LANDSAT
snic = ee.Algorithms.Image.Segmentation.SNIC(
  image = image_corrected_composite_landsat, 
  size=4,
  compactness=2,
  connectivity=4,
  neighborhoodSize=8,
  seeds=seeds
).select(['red_mean', 'green_mean', 'blue_mean', 'nir_mean', 'swir1_mean', 'swir2_mean', 'clusters'],['red', 'green', 'blue', 'nir', 'swir1' ,'swir2','clusters'])

clusters = snic.select('clusters')
# Map.addLayer(clusters.randomVisualizer(), {}, 'clusters')

In [17]:
Map.addLayer(snic, {"bands": ['swir2', 'nir', 'red'], min:0, max:1, "gamma": 0.8}, 'means')

In [18]:
classImageSpectral_snic = SpectralAnalysis(snic, AOI = AOI, I_satellite= 'LANDSAT', pca_scaling = 1, tileScale = 1)


In [19]:
avi_image_2 = classImageSpectral_snic.AVI_func()

In [20]:
bsi_image = classImageSpectral_snic.BSI_func()

si_image = classImageSpectral_snic.SI_func()

In [21]:
# generated the existing pca_scale based on the class assigned
pca_scale = classImageSpectral_snic.pca_scale

In [22]:
from osi.spectral_indices.utils import normalization_100

avi_norm = normalization_100(avi_image_2, pca_scale=pca_scale, AOI=AOI)

In [23]:
Map.addLayer(avi_image_2,{},'avi_2')

In [24]:
Map.addLayer(avi_norm,{'min':0,'max':100}, 'avi_norm')

In [25]:
bsi_norm = normalization_100(bsi_image, pca_scale=pca_scale, AOI=AOI)
Map.addLayer(bsi_norm,{'min':0,'max':100},'bsi_norm')
print('Normalizing to 100 SI')
si_norm = normalization_100(si_image, pca_scale=pca_scale, AOI=AOI)
Map.addLayer(si_norm, {'min':0,'max':100},'si_norm')
print('Combining AVI AND BSI')


Normalizing to 100 SI
Combining AVI AND BSI


In [26]:
# Combine  AVI and BSI to one image with two bands 
AVI_BSI = avi_norm.addBands(bsi_norm)
# Masked-out process or remove null data, to avoid errors
avi_bsi_clean = AVI_BSI.gte(0).Or(AVI_BSI.lte(0))
AVI_BSI = AVI_BSI.updateMask(avi_bsi_clean)

In [27]:
#### PRINCIPAL COMPONENT ANALYSIS ########
# adapted from https://developers.google.com/earth-engine/guides/arrays_eigen_analysis
# Define the mean-centered function
def means_centered(image, region, scale, tileScale):
    bandNames = image.bandNames()

    # Mean center the data
    meanDict = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region,
        scale=scale,
        maxPixels=1e9,
        tileScale=tileScale,
        bestEffort=True,
    )
    means = ee.Image.constant(meanDict.values(bandNames))
    centered = image.subtract(means)

    return [centered, bandNames]

# Define the helper function to generate new band names
def getNewBandNames(prefix, bandNames):
    seq = ee.List.sequence(1, bandNames.length())
    return seq.map(lambda b: ee.String(prefix).cat(ee.Number(b).int().format()))

# Define the function to get principal components
def getPrincipalComponents(centered, scale, region, bandNames):
    # Collapse the bands into a 1D array per pixel
    arrays = centered.toArray()

    # Compute the covariance within the region
    covar = arrays.reduceRegion(
        reducer=ee.Reducer.centeredCovariance(),
        geometry=region,
        scale=scale,
        maxPixels=1e9,
        tileScale=tileScale,
        bestEffort=True,
        
    )

    # Get the covariance array result and cast to an array
    covarArray = ee.Array(covar.get('array'))

    # Perform eigen analysis and slice apart the values and vectors
    eigens = covarArray.eigen()

    # Extract eigenvalues and eigenvectors
    eigenValues = eigens.slice(1, 0, 1)
    eigenVectors = eigens.slice(1, 1)

    # Convert the array image to 2D arrays
    arrayImage = arrays.toArray(1)

    # Left multiply the image array by the matrix of eigenvectors
    principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage)

    # Square root of eigenvalues as a P-band image
    sdImage = ee.Image(eigenValues.sqrt()) \
        .arrayProject([0]).arrayFlatten([getNewBandNames('sd', bandNames)])

    # Normalize the principal components by their standard deviations
    return principalComponents \
        .arrayProject([0]) \
        .arrayFlatten([getNewBandNames('pc', bandNames)]) \
        .divide(sdImage)

In [28]:
tileScale = classImageSpectral_snic.tileScale

In [29]:
print('Processing means center of AVI_BSI please wait')
# Means Centered for VD --- VD WILL GIVE YOU ERROR ---> if you dont masked out first
Means_cn = means_centered(AVI_BSI, AOI, pca_scale, tileScale)
#print('Means_cn[0]',Means_cn[0])
#print('Means_cn[1]',Means_cn[1])

print('Now we proceed to the PCA of Vegetation density')
# PCA FOR AVI AND BSI = VD and SVI
VD = getPrincipalComponents(Means_cn[0], pca_scale, AOI, Means_cn[1]).rename(['VD1', 'VD2'])
#Map.addLayer(VD,{},'VD')
# scale VD datasets 
SVI = normalization_100(VD,pca_scale=pca_scale, AOI=AOI)
Map.addLayer(SVI,{'min':0,'max':100},'SVI', False)
print('Success get the PCA normalized of VD => SVI')

Processing means center of AVI_BSI please wait
Now we proceed to the PCA of Vegetation density
Success get the PCA normalized of VD => SVI


In [30]:
Map

Map(center=[-1.0750537908647946, 102.44795563556586], controls=(WidgetControl(options=['position', 'transparen…